1. Experimenten con el numero de neuronas en el modelo al igual que el learning rate.
- Que cambios resultan en un output mas lineal del modelo?
- Pueden hacer que el modelo haga un overfit obvio de la data?



2. Cargen la data de vinos blancos y creen un modelo con el numero apropiado de inputs

- Cuanto tarda en entrenar comparado al dataset que hemos estado usando?
- Pueden explicar que factores contribuyen a los tiempos de entrenamiento?
- Pueden hacer que el loss disminuya?
- Intenten graficar la data

In [1]:
import torch
from torch import nn
import torch.optim as optim
import numpy as np
from os import listdir
from os.path import isfile, join


In [2]:
#Clases y Funciones


def training_loop(model, n_epochs, optimizer, loss_fn, train_x, val_x, train_y, val_y):
    for epoch in range(1, n_epochs + 1):
        train_t_p = model(train_x) # ya no tenemos que pasar los params
        train_loss = loss_fn(train_t_p, train_y)
        
        with torch.no_grad(): # todos los args requires_grad=False
            val_t_p = model(val_x)
            val_loss = loss_fn(val_t_p, val_y)
        
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        if epoch == 1 or epoch % 1000 == 0:
          print(f"Epoch {epoch}, Training loss {train_loss}, Validation loss {val_loss}")
            
class SubclassFunctionalModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden_linear = nn.Linear(1, 14)
        self.output_linear = nn.Linear(14, 1)

        
    def forward(self, input):
        hidden_t = self.hidden_linear(input)
        activated_t = torch.tanh(hidden_t)
        output_t = self.output_linear(activated_t)

        return output_t

class SubclassModel(nn.Module):
    def __init__(self, num):
        super().__init__()
        self.hidden_linear = nn.Linear(1, num)
        self.hidden_activation = nn.Tanh()
        self.output_linear = nn.Linear(num, 1)

    def forward(self, input):
        hidden_t = self.hidden_linear(input)
        activated_t = self.hidden_activation(hidden_t)
        output_t = self.output_linear(activated_t)

        return output_t


In [3]:

t_c = [0.5, 14.0, 15.0, 28.0, 11.0, 8.0, 3.0, -4.0, 6.0, 13.0, 21.0] # Temperatura en grados celsios
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4] # Unidades desconocidas
t_c = torch.tensor(t_c).unsqueeze(1) # Agregamos una dimension para tener B x N_inputs
t_u = torch.tensor(t_u).unsqueeze(1) # Agregamos una dimension para tener B x N_inputs

n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_t_u = t_u[train_indices]
train_t_c = t_c[train_indices]

val_t_u = t_u[val_indices]
val_t_c = t_c[val_indices]

train_t_un = 0.1 * train_t_u
val_t_un = 0.1 * val_t_u

In [4]:
############################################Intento 1####################################################
subclass_model = SubclassModel(50)

optimizer = optim.SGD(subclass_model.parameters(), lr=1e-5)
training_loop(
    n_epochs=5000,
    optimizer=optimizer,
    model=subclass_model,
    loss_fn=nn.MSELoss(), # Ya no estamos usando nuestra loss function hecha a mano
    train_x = train_t_un,
    val_x = val_t_un,
    train_y = train_t_c,
    val_y = val_t_c)

print('output', subclass_model(val_t_un))
print('answer', val_t_c)
print('hidden', subclass_model.hidden_linear.weight.grad)

Epoch 1, Training loss 227.1898651123047, Validation loss 137.93728637695312
Epoch 1000, Training loss 104.94120788574219, Validation loss 32.02037811279297
Epoch 2000, Training loss 78.16907501220703, Validation loss 15.62597942352295
Epoch 3000, Training loss 70.80577087402344, Validation loss 14.734235763549805
Epoch 4000, Training loss 66.78810119628906, Validation loss 15.233122825622559
Epoch 5000, Training loss 63.281982421875, Validation loss 15.277766227722168
output tensor([[10.9715],
        [11.5835]], grad_fn=<AddmmBackward>)
answer tensor([[ 6.],
        [14.]])
hidden tensor([[-0.2874],
        [ 1.4643],
        [-0.1928],
        [-1.5301],
        [ 0.6822],
        [-0.4208],
        [ 0.7170],
        [ 2.1167],
        [-0.6926],
        [ 0.2864],
        [-0.5172],
        [ 1.7126],
        [ 2.0038],
        [ 0.0080],
        [ 0.1402],
        [-0.0434],
        [ 0.4847],
        [ 0.1191],
        [ 0.1977],
        [-1.5419],
        [ 0.8552],
        [-1

In [5]:
############################################Intento 2####################################################
subclass_model = SubclassModel(1000)

optimizer = optim.SGD(subclass_model.parameters(), lr=1e-5)
training_loop(
    n_epochs=5000,
    optimizer=optimizer,
    model=subclass_model,
    loss_fn=nn.MSELoss(), # Ya no estamos usando nuestra loss function hecha a mano
    train_x = train_t_un,
    val_x = val_t_un,
    train_y = train_t_c,
    val_y = val_t_c)

print('output', subclass_model(val_t_un))
print('answer', val_t_c)
print('hidden', subclass_model.hidden_linear.weight.grad)

Epoch 1, Training loss 203.8484649658203, Validation loss 116.9561538696289
Epoch 1000, Training loss 55.82001495361328, Validation loss 15.818521499633789
Epoch 2000, Training loss 40.11280822753906, Validation loss 14.70352840423584
Epoch 3000, Training loss 29.22624969482422, Validation loss 13.790350914001465
Epoch 4000, Training loss 21.88345718383789, Validation loss 13.021085739135742
Epoch 5000, Training loss 17.05974578857422, Validation loss 12.361532211303711
output tensor([[10.9360],
        [13.4020]], grad_fn=<AddmmBackward>)
answer tensor([[ 6.],
        [14.]])
hidden tensor([[-2.5529e-02],
        [-1.6419e-01],
        [-1.0247e-02],
        [-2.0078e-02],
        [ 4.1668e-03],
        [-5.1272e-03],
        [ 7.0656e-05],
        [ 8.9424e-02],
        [ 4.4134e-03],
        [-2.0467e-01],
        [ 6.1034e-02],
        [-6.3968e-03],
        [-1.3632e-05],
        [ 3.8927e-02],
        [ 2.4013e-03],
        [-7.8301e-05],
        [-7.3741e-01],
        [ 3.4150e-

In [6]:
############################################Intento 3####################################################
subclass_model = SubclassModel(120)

optimizer = optim.SGD(subclass_model.parameters(), lr=1e-2)
training_loop(
    n_epochs=5000,
    optimizer=optimizer,
    model=subclass_model,
    loss_fn=nn.MSELoss(), # Ya no estamos usando nuestra loss function hecha a mano
    train_x = train_t_un,
    val_x = val_t_un,
    train_y = train_t_c,
    val_y = val_t_c)

print('output', subclass_model(val_t_un))
print('answer', val_t_c)
print('hidden', subclass_model.hidden_linear.weight.grad)

Epoch 1, Training loss 198.5008544921875, Validation loss 111.4076919555664
Epoch 1000, Training loss 11.112096786499023, Validation loss 16.463834762573242
Epoch 2000, Training loss 4.429706573486328, Validation loss 5.2980875968933105
Epoch 3000, Training loss 2.343747615814209, Validation loss 3.381012201309204
Epoch 4000, Training loss 1.7161966562271118, Validation loss 3.2322564125061035
Epoch 5000, Training loss 1.5252286195755005, Validation loss 3.340428590774536
output tensor([[ 7.1029],
        [11.2233]], grad_fn=<AddmmBackward>)
answer tensor([[ 6.],
        [14.]])
hidden tensor([[ 2.0810e-01],
        [ 2.2465e-01],
        [-1.0361e+00],
        [-2.7257e-01],
        [ 2.7322e-01],
        [-1.2314e-01],
        [ 8.2893e-01],
        [ 8.1940e-01],
        [-2.7229e-01],
        [-3.9976e-01],
        [ 7.6426e-01],
        [ 7.9184e-01],
        [ 9.6917e-01],
        [ 1.0460e+00],
        [-1.8719e-01],
        [ 1.4252e-01],
        [ 1.2002e+00],
        [-9.1400

In [7]:
#################################### Respuesta 1##############################################################

#Asi como se puede ver en el intento 1, el tener un learning rate mas pequeño hace que sea necesario aumenta el numero de epochs para que el loss se reduzca 
#En el intento 3 probamos que el loss se puede reducir aumentando el numero de epochs y aumentando el learning rate, pero esto tiende a un overfit de la data, por lo que decidir
#cuidadosamente cuantos epochs tendra la funcion y el porcentaje de learning rate. 

In [10]:
#################################### Intento 4 ##############################################################
import csv
import csv
with open('winequality-white.csv', 'r') as f:
    wines = list(csv.reader(f, delimiter=';'))


OSError: ignored